In [ ]:
import os
import glob
import json
from dateutil import parser
import pandas as pd
import numpy as np

In [ ]:
files = [os.path.basename(x) for x in glob.glob('./un_users/*.json')]

In [ ]:
#dt = pd.DataFrame()
for file_name in files:
    f = open('./users/%s'%file_name)
    context = f.readlines()
    data = []
    for i in context[:]:
        data += [json.loads(i.replace(',\n','').replace('\n',''))]
        
    uIds = []
    uIds_ind = []
    fPs = []
    fPs_ind = []
    ips = []
    ips_ind = []
    #print files.index(file_name)
    for field_name, list_mass, ind_mass in [('userId', uIds, uIds_ind), ('fingerPrint', fPs, fPs_ind), ('ip', ips, ips_ind)]:
        try:
            list_mass += [i[field_name] for i in data]
            temp_set = list(set(list_mass))
            if len(temp_set)>1:
                ind_mass += sorted([list_mass.index(i) for i in temp_set])
            else:
                ind_mass += [None]
        except KeyError:
            list_mass += [None]
            ind_mass += ['empty']
            pass
    deltas = [0]
    times = [parser.parse(i['timestamp']) for i in data]
    deltas += [(times[i] - times[i-1]).total_seconds() for i in range(1, len(times))]
    dt=dt.append({
            'UserIds': len(set(uIds)),
            'UserIds_ind': uIds_ind,
            'fingerPrint': len(set(fPs)),
            'fingerPrint_ind': fPs_ind,
            'ips': len(set(ips)),
            'ips_ind': ips_ind,
            'deltas': deltas
        }, ignore_index=True)
    print len(dt)

In [ ]:
#reg_users
f = open('files_reg_users.txt', 'w')
ar1 = dt[dt['ips']==1][dt['fingerPrint'] ==1][dt['UserIds']==2].index
for i in ar1:
    f.write(files[i]+'\n')
f.close()

In [ ]:
hours = 12
f = open('files_slicable_users_by_ip.txt', 'w')
ar2 = []
for k in dt[dt['ips']>1][dt['fingerPrint']==1].index.values:
    inds = sorted([i for i in dt.iloc[k]['ips_ind'] if i !=0])
    flag = True
    for i in inds:
        if not abs(dt.iloc[k]['deltas'][i]) > 3600*hours:
            flag = False
            break
    if flag:
        #print k
        ar2 += [k]
        f.write(files[k]+'\n')
print len(ar2)
f.close()

In [ ]:
hours = 12
f = open('files_slicable_users_by_fP.txt', 'w')
ar3 = []
for k in dt[dt['ips']==1][dt['fingerPrint']>1].index.values:
    inds = sorted([i for i in dt.iloc[k]['fingerPrint_ind'] if i !=0])
    if inds == [None]:
        continue
    flag = True
    for i in inds:
        if not abs(dt.iloc[k]['deltas'][i]) > 3600*hours:
            flag = False
            break
    if flag:
        #print k
        ar3 += [k]
        f.write(files[k]+'\n')
print len(ar3)
f.close()

In [ ]:
total = sorted(list(set(list(ar1)+ar2+ar3)))

In [ ]:
f=open('files_unun_users.txt','w')
for i in range(0,len(files)):
    if i not in total:
        f.write(files[i]+'\n')
f.close()

In [ ]:
dt.to_csv('46000.csv')